# NHS Table Parser

## Import from CSV

In [23]:
import pandas as pd

class importCSV:
    # imports a csv and returns a pandas table
    def __init__(self, CSVfile, output):
        # given it's Python 2.7 it has to check encoding
        rawTable = False
        try:
            rawTable = [line.strip().decode('latin-1').split(',') for line in open(CSVfile, 'r')]

        except UnicodeDecodeError:
            rawTable = [line.strip().decode('utf-8').split(',') for line in open(CSVfile, 'r')]

        except IOError as e:
            print 'Could not open %s' %(CSVfile)

        self.data = False
        if rawTable:
            matrix = []
            maximumLen = max(len(row) for row in rawTable)
            for i, row in enumerate(rawTable):
                if len(row) == maximumLen:
                    matrix.append(row)
            # data object is now a pandas table
            self.data = pd.DataFrame(matrix[1:], columns=matrix[0])

In [21]:
csvf = importCSV('07_11_france.csv')

In [5]:
csvf.data.to_csv()

## Import from Excel

In [107]:
import xlrd
import pandas as pd
from dateutil.parser import parse

class importXLS:
    
    def __init__(self, xlsFile, filePath):
        # import an Excel file
        
        try:
            excel = xlrd.open_workbook(xlsFile)
            sheet_names = excel.sheet_names()
            self.data = [self.parseSheet(excel.sheet_by_name(key), key) for key in excel.sheet_names()]
            for table in self.data:
                if not filePath.endswith('/') and filePath:
                    filePath = filePath.strip() + '/'
                dateInFile = self.checkIfDate(xlsFile)
                try:
                    dataName = table.keys()[0]
                    if dateInFile:
                        table[dataName]['table'].to_csv(filePath + dataName.replace(' ', '_') + '_' + dateInFile + '.csv')
                        continue
                    table[dataName]['table'].to_csv(filePath + dataName.replace(' ', '_') + '_' + table[dataName]['date'] + '.csv')
                except Exception as e:
                    print 'Error writing table to CSV: %s' %(e)
        except Exception as e:
            print 'Error parsing Excel file: %s' %(e)
                
    def parseSheet(self, sheet, sheetName):
        # parses the excel sheet
        
        # the data is returned as a dict in order to return the date
        dataReturn = {sheetName: {
                'date': '',
                'table': False
            }}
        def exists(n):
            if n:
                if type(n) == str:
                    if n.strip():
                        return True
                    return False
                return True
                
            return False
        
        try:
            values = [[sheet.cell(row, col).value for col in range(0, sheet.ncols)] for row in range(0, sheet.nrows)]
            # initial loop to find the table and skip the headers
            rowLengths = []
            longRowCounter = 0
            tableStarts = {'row': 0, 'col': 0}
            superfields = False
            for i, row in enumerate(values):
                realRowLength = len([n for n in row if exists(n)])
                if not realRowLength > 0:
                    continue
                rowLengths.append(realRowLength)
                # check if the row could be in the table
                if realRowLength == max(rowLengths):
                    longRowCounter += 1
                # once the lengths are long and consistent then we are reading the table
                if longRowCounter > 20:
                    # we found the table, now lets find the column it starts in
                    tableStarts['col'] = (i for i,v in enumerate(row) if exists(v)).next()
                    break
            # second loop to find the row number where the table starts
            for i, row in enumerate(values):
                realRowLength = len([n for n in row if exists(n)])
                if realRowLength == max(rowLengths):
                    tableStarts['row'] = i
                    break
            # just to be safe lets check if there are any "super" fields
            if len(values[tableStarts['row'] - 1]) > min(rowLengths):
                # there are indeed "super" fields
                superfields = True
                tableStarts['row'] -= 1
            # now return the initial header to find the datestamp
            init_header = values[:tableStarts['row']]
            dataReturn[sheetName]['date'] = self.findDate(init_header)
            # build the table
            matrix = []
            endReached = False
            # clean up table
            for i, row in enumerate(values[tableStarts['row']:]):
                realRowLength = len([n for n in row if exists(n)])
                # check if table finished
                if realRowLength < 2:
                    emptyRowCount = 0
                    nextRowCount = 0
                    for nextRow in values[i:]:
                        if len([n for n in nextRow if exists(n)]) < 2:
                            emptyRowCount += 1
                            if emptyRowCount > 20:
                                endReached = True
                                emptyRowCount = 0
                                break
                            continue
                        nextRowCount += 1
                        if nextRowCount > 20:
                            break
                if endReached:
                    break
                            
                matrix.append(row[tableStarts['col']:])
            header = self.sortHeader(matrix, superfields)
            # check if there's a totals row - which is most likely
            table = pd.DataFrame(matrix[4:], columns=header) if superfields else pd.DataFrame(matrix[3:], columns=header)
            table.dropna(axis=1, how='all')
            table.dropna(axis=0, how='all')
            dataReturn[sheetName]['table'] = table

            return dataReturn
            
        except Exception as e:
            print 'Error parsing sheet: %s' %(e)
            
        return []
    
    def sortHeader(self, matrix, superfields):
        # will sort out the header
        
        if superfields:
            header = matrix[1]
            currentSuperField = ''
            for col, field in enumerate(header):
                if matrix[0][col]:
                    currentSuperField = matrix[0][col]
                    field = field + ' | ' + currentSuperField
                header[col] = field
            return header
        return matrix[0]
    
    def checkIfDate(self, fileName):
        # checks if the date is in the inital file name
        
        head, sep_, tail = fileName.partition('/')
        fileName = tail
        for sep in ['-', '_']:
            for bit in fileName.split(sep):
                try:
                    date = parse(bit)
                    return date.strftime('%Y-%m-%d')
                except ValueError:
                    pass
        return False
    
    def findDate(self, header):
        # finds the published date
        
        for line in header:
            for i, value in enumerate(line):
                if value:
                    if 'period' in value.lower():
                        # next cell is the date
                        try:
                            date = parse(line[i + 1])
                            return date.strftime('%Y-%m-%d')
                        except ValueError:
                            pass
        return ''

In [95]:
excel = importXLS('May-2017-AE-by-provider-4ZNN7.xls', 'output')

## Loop through all the files and output the CSVs

In [4]:
from os import listdir

for fileName in listdir('inputFiles/'):
    excel = importXLS('inputFiles/' + fileName, 'output')

## Upload to S3

In [3]:
import os

import boto
import boto.s3
from boto.s3.key import Key


bucket_name = "nhspublicdata"

#
# Get keys from env
#
AWS_SECRET_KEY='xxxx'
AWS_ACCESS_KEY='xxx'

def upload_to_s3( filename, bucket ) :
    c = boto.connect_s3(AWS_ACCESS_KEY, AWS_SECRET_KEY)
    b = c.get_bucket( bucket_name )

    k = Key(b)
    k.key = os.path.basename( filename )
    k.set_contents_from_filename( filename )


if __name__ == "__main__" :
    for fileName in os.listdir('output'):
        upload_to_s3( fileName, bucket_name )
